In [2]:
import cv2 
import mediapipe as mp
import time
import numpy as np
import cvzone.HandTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities , IAudioEndpointVolume


In [3]:
cap=cv2.VideoCapture(0)
detector=htm.HandDetector(detectionCon=0.7) #to be more precise in hand tracking
#to display the frame hjjfo
cTime=0
pTime=0
#volume parameters
devices = AudioUtilities.GetSpeakers()
interface=devices.Activate(IAudioEndpointVolume._iid_ , CLSCTX_ALL , None)
volume=cast(interface , POINTER(IAudioEndpointVolume))
volume.GetMute()
volume.GetMasterVolumeLevel()

volumeRange=volume.GetVolumeRange()
minVol=volumeRange[0]
maxVol=volumeRange[1]
volBar=400


In [5]:
while 1 : 
    
    succ, img= cap.read()
    img=detector.findHands(img)
    #to find info and positions
    lmListe = detector.findPosition(img, draw=False) 
    #find position returns a list that contains different positions of the hand in the hole pic
     
    
    if len(lmListe[0])>=3:  
        x1,y1=lmListe[0][4][0] ,  lmListe[0][4][1]
        x2,y2=lmListe[0][8][0] ,  lmListe[0][8][1]
        #get the middle of  the line 
        cx, cy = (x1+x2)//2 , (y1+y2)//2
        
        #draw a circle on the desired index : 4 and 8
        cv2.circle(img,(x1,y1),10,(255,0,255),cv2.FILLED)
                              #10:radious of the circle
        cv2.circle(img,(x2,y2),10,(255,0,255),cv2.FILLED)
        cv2.line(img,(x1,y1) ,(x2,y2) ,(255,0,255) , 2)
                                                    #2: thikness of the line
        
        lenght=math.hypot(x2-x1 , y2-y1)
        # we have to convert hand range [50..300]  to volume Range [-65.0]=>np.interp()
        vol=np.interp(lenght, [50,300], [minVol , maxVol])

        volBar=np.interp(lenght, [50,300], [400,150])
        #to set the master volume
        volume.SetMasterVolumeLevel(vol, None)   #retrun the intervall of volume

    #draw a volume rec 
    cv2.rectangle(img, (50,150),(80,400),(255,0,0), 3)
    cv2.rectangle(img, (50,int(volBar)),(80,400),(255,0,0), cv2.FILLED)
    
    #we can display the frame rate : 
    cTime = time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    cv2.putText(img, str(int(fps)),(10,70), 
                cv2.FONT_HERSHEY_PLAIN, 3 , (255,0,255),3)


    #to display the window
    cv2.imshow('img', img)
    cv2.waitKey(10)
    if cv2.waitKey(10)==ord('a'):
        break
cv2.destroyAllWindows()